In [7]:
import time
from pathlib import Path

import numpy as np

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

rng_seed = 42

print(f"Data will be saved to {data_dir.resolve()}")
print(f"Random seed is {rng_seed}")

Data will be saved to /Users/Tonni/Desktop/master-code/neural-quantum-tomo/case_studies/w_phase_augmented_old/data
Random seed is 42


In [ ]:
"""
In-place Pauli {Z,X,Y} measurement on the two slices of a qubit.
Modifies plus_slice (for |0⟩ component) and minus_slice (for |1⟩ component) in place.

Docstring convention from original: Returns 0 for +1 outcome (‘0’), 1 for −1 (‘1’).
NOTE: The original code (and this revised version, to match behavior)
has a specific convention for the 'Y' operator's returned bit:
- Outcome 0 is associated with projection onto a state proportional to |0> - i*|1> (the -1 eigenstate |-i>).
- Outcome 1 is associated with projection onto a state proportional to |0> + i*|1> (the +1 eigenstate |+i>).
This means for 'Y', outcome 0 corresponds to eigenvalue -1, and outcome 1 to eigenvalue +1,
which is inverted compared to the Z and X operators' outcome/eigenvalue mapping.

This version also corrects a bug in the original 'X' and 'Y' probability
calculations (must use np.abs(amplitude)**2 for complex amplitudes).
"""

In [8]:
def measure_one(plus_slice, minus_slice, op, rng):

    if op not in ('I', 'Z', 'X', 'Y'):
        raise ValueError(f"Unsupported Pauli operator: {op}. Expected 'I', 'X', 'Y', or 'Z'.")

    if op == 'I':
        # the identity operator does not overwrite the slices and thus lets the state unchanged
        # we return 0 for the +1 eigenvalue as an arbitrary convention
        return 0

    INV_SQRT2 = 1.0 / np.sqrt(2.0)

    plus_slice_tmp = plus_slice.copy()
    minus_slice_tmp = minus_slice.copy()

    # initialize with the Z basis
    plus_ampl = plus_slice_tmp
    minus_ampl = minus_slice_tmp

    plus_eigvec = (1.0, 0.0)
    minus_eigvec = (0.0, 1.0)


    if op == 'X':
        plus_ampl = (plus_slice_tmp + minus_slice_tmp) * INV_SQRT2
        minus_ampl = (plus_slice_tmp - minus_slice_tmp) * INV_SQRT2

        plus_eigvec = (INV_SQRT2, INV_SQRT2)
        minus_eigvec = (INV_SQRT2, -INV_SQRT2)


    if op == 'Y':
        plus_ampl = (plus_slice_tmp + 1j * minus_slice_tmp) * INV_SQRT2
        minus_ampl = (plus_slice_tmp - 1j * minus_slice_tmp) * INV_SQRT2

        plus_eigvec = (INV_SQRT2, -1j * INV_SQRT2)
        minus_eigvec = (INV_SQRT2, 1j * INV_SQRT2)


    # we square up all amplitudes where we have 0 or 1 at the current index and sum them up
    plus_prob = np.sum(np.abs(plus_ampl)**2)
    plus_prob = np.clip(plus_prob, 0.0, 1.0)
    minus_prob = 1.0 - plus_prob
    plus_meas = rng.random() < plus_prob


    chosen_ampl = (plus_ampl if plus_meas else minus_ampl)
    chosen_eigvec = (plus_eigvec if plus_meas else minus_eigvec)
    renorm_factor = np.sqrt(plus_prob if plus_meas else minus_prob)


    # in case the is tiny, we need to do some special handling
    # renorm_factor == 0 implies chosen_ampl is an array of zeros (assuming a valid normalized initial state).
    if renorm_factor < 1e-12:
        plus_slice[:] = 0.0
        minus_slice[:] = 0.0

        return 0 if plus_meas else 1


    plus_slice[:] = (chosen_ampl * chosen_eigvec[0]) / renorm_factor
    minus_slice[:] = (chosen_ampl * chosen_eigvec[1]) / renorm_factor
    return 0 if plus_meas else 1


def sample_state(state_vector, pauli_ops, rng):
    num_qubits = len(pauli_ops)

    # reshape the state vector in place to allow for multi-dimensional indexing
    state_tensor = state_vector.reshape((2,) * num_qubits)
    outcome_bits = []

    for qubit_idx, pauli_op in enumerate(pauli_ops):
        # we create only a view of the state tensor to be modified in place by the measure_one function
        plus_slice  = state_tensor[(slice(None),) * qubit_idx + (0,)]
        minus_slice = state_tensor[(slice(None),) * qubit_idx + (1,)]

        outcome_bit = measure_one(plus_slice, minus_slice, pauli_op, rng)
        outcome_bits.append(outcome_bit)

    return outcome_bits

In [13]:
def format_bytes(num_bytes):
    """Utility function to format bytes into KB, MB, GB, etc."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if num_bytes < 1024.0:
            return f"{num_bytes:.2f} {unit}"
        num_bytes /= 1024.0
    return f"{num_bytes:.2f} PB"


# constructing the phase augmented w state

num_qubits = 10

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
w_aug = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    w_aug[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)


print(f"Size of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

for i in range(10):
    print(f"{i:0{num_qubits}b}: {w_aug[i]:.2f} + {w_aug[i].imag:.2f}j")

Size of state vector in memory: 16.00 KB 

0000000000: 0.00+0.00j + 0.00j
0000000001: -0.30+0.10j + 0.10j
0000000010: 0.22+0.23j + 0.23j
0000000011: 0.00+0.00j + 0.00j
0000000100: 0.07-0.31j + -0.31j
0000000101: 0.00+0.00j + 0.00j
0000000110: 0.00+0.00j + 0.00j
0000000111: 0.00+0.00j + 0.00j
0000001000: 0.02-0.32j + -0.32j
0000001001: 0.00+0.00j + 0.00j


In [14]:
import numpy as np
import time
from pathlib import Path
from functools import reduce # For np.kron over a list

# Pauli matrices (standard definitions)
pauli = {
    'I': np.array([[1, 0], [0, 1]], dtype=complex),
    'X': np.array([[0, 1], [1, 0]], dtype=complex),
    'Y': np.array([[0, -1j], [1j, 0]], dtype=complex),
    'Z': np.array([[1, 0], [0, -1]], dtype=complex)
}

def get_theoretical_expectation(state_vector, pauli_string_tuple):
    """
    Calculates the theoretical expectation value <state|P|state>
    where P is the tensor product of Pauli operators in pauli_string_tuple.
    """
    num_qubits_in_op = len(pauli_string_tuple)
    state_dim_from_vec = state_vector.shape[0]

    # Infer num_qubits from state_vector dimension
    num_qubits_from_state = 0
    if state_dim_from_vec > 0:
        num_qubits_from_state = int(np.log2(state_dim_from_vec))
        if (1 << num_qubits_from_state) != state_dim_from_vec: # Check if power of 2
            raise ValueError(f"State vector dimension {state_dim_from_vec} is not a power of 2.")

    if num_qubits_in_op != num_qubits_from_state:
        raise ValueError(f"Pauli string length {num_qubits_in_op} does not match inferred qubits from state vector {num_qubits_from_state}.")

    op_list = [pauli[op_char] for op_char in pauli_string_tuple]
    multi_qubit_op = reduce(np.kron, op_list)

    expectation = np.dot(state_vector.conj(), np.dot(multi_qubit_op, state_vector))

    if np.abs(np.imag(expectation)) > 1e-9:
        print(f"Warning: Theoretical expectation for {''.join(pauli_string_tuple)} has non-negligible imaginary part: {expectation.imag}")
    return np.real(expectation)

def get_observed_expectation_from_sample_state(state_vector, pauli_string_tuple, num_samples, rng_seed_for_sampling, sample_state_func, measure_one_func):
    """
    Calculates the observed expectation value by sampling using the provided sample_state function.
    """
    num_qubits_in_op = len(pauli_string_tuple)

    eigenvalue_maps = {
        'I': {0: 1, 1: 1},
        'X': {0: 1, 1: -1},
        'Y': {0: -1, 1: 1},
        'Z': {0: 1, 1: -1}
    }

    sum_of_sample_eigenvalues = 0.0
    local_rng = np.random.default_rng(rng_seed_for_sampling)

    for op_char in pauli_string_tuple:
        if op_char == 'I':
            # Note: The user's measure_one does not handle 'I'.
            # This verification should ideally use pauli_string_tuples without 'I'
            # or sample_state_func must be adapted for 'I'.
            # If sample_state_func passes 'I' to measure_one_func, it will raise ValueError.
            pass # Assume sample_state_func handles 'I' correctly if present.

    for _ in range(num_samples):
        state_vector_copy = state_vector.copy()
        outcome_bits = sample_state_func(state_vector_copy, pauli_string_tuple, local_rng)

        current_sample_eigenvalue = 1.0
        for i in range(num_qubits_in_op):
            op_char = pauli_string_tuple[i]
            outcome_bit = outcome_bits[i]
            current_sample_eigenvalue *= eigenvalue_maps[op_char][outcome_bit]
        sum_of_sample_eigenvalues += current_sample_eigenvalue

    return sum_of_sample_eigenvalues / num_samples

# Assuming measure_one, sample_state, num_qubits, w_aug, rng_seed are defined
# in the global scope of your notebook before this cell.

print("Verification helper functions defined.")
print("Note: The `sample_state` function in the notebook modifies the input state_vector. "
      "Copies will be used for each sampling run in the verification.")

Verification helper functions defined.
Note: The `sample_state` function in the notebook modifies the input state_vector. Copies will be used for each sampling run in the verification.


In [15]:
# Ensure num_qubits, w_aug, rng_seed, measure_one, sample_state are available from previous cells.

# Verification parameters
num_samples_for_verification = 10000

# Define some Pauli strings to test (length must match num_qubits)
# These strings should NOT contain 'I' if using the provided measure_one directly.
if num_qubits == 10:
    pauli_strings_to_test = [
        ('Z',) * num_qubits,
        ('X',) * num_qubits,
        ('Y',) * num_qubits,
        ('Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z'),
        ('X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X'),
    ]
    if num_qubits >= 2: # Always true for num_qubits = 10
        mixed_ops_list = ['X'] * num_qubits
        mixed_ops_list[0] = 'Z'
        mixed_ops_list[1] = 'Z'
        pauli_strings_to_test.append(tuple(mixed_ops_list))

        single_z_op_as_xs_background = ['X'] * num_qubits
        single_z_op_as_xs_background[0] = 'Z'
        pauli_strings_to_test.append(tuple(single_z_op_as_xs_background))

        single_y_op_as_xs_background = ['X'] * num_qubits
        single_y_op_as_xs_background[num_qubits // 2] = 'Y'
        pauli_strings_to_test.append(tuple(single_y_op_as_xs_background))

elif num_qubits == 3:
    pauli_strings_to_test = [
        ('Z', 'Z', 'Z'),
        ('X', 'X', 'X'),
        ('Y', 'Y', 'Y'),
        ('Z', 'X', 'Y'),
    ]
else:
    print(f"No predefined test strings for num_qubits = {num_qubits}. Please add some or adjust.")
    pauli_strings_to_test = []

print(f"--- Verifying for {num_qubits}-qubit w_aug state ---")
# rng_seed from the notebook is for generating w_aug phases.
# For the sampling part of *this verification*, use a distinct or derived seed.
verification_sampling_seed = rng_seed + 456 # Example of a derived seed
print(f"Seed for w_aug generation (from notebook): {rng_seed}")
print(f"Seed for verification sampling runs: {verification_sampling_seed}")
print(f"Number of samples per setting for verification: {num_samples_for_verification}\n")

results = []

if not pauli_strings_to_test:
    print("No Pauli strings defined for testing. Exiting verification.")
else:
    for ps_tuple in pauli_strings_to_test:
        ps_str = "".join(ps_tuple)
        print(f"Testing Pauli String: {ps_str}")

        theo_exp = get_theoretical_expectation(w_aug, ps_tuple)
        print(f"  Theoretical Expectation: {theo_exp:.6f}")

        obs_exp = get_observed_expectation_from_sample_state(
            w_aug,
            ps_tuple,
            num_samples_for_verification,
            verification_sampling_seed,
            sample_state, # function from notebook
            measure_one   # function from notebook
        )
        print(f"  Observed Expectation:    {obs_exp:.6f}")

        diff = abs(theo_exp - obs_exp)
        print(f"  Absolute Difference:     {diff:.6f}\n")
        results.append({'string': ps_str, 'theo': theo_exp, 'obs': obs_exp, 'diff': diff})

    print("--- Summary of Differences ---")
    header_len = num_qubits if num_qubits > 0 else 10
    for res in results:
        print(f"{res['string']:<{header_len}} : Theo={res['theo']:.4f}, Obs={res['obs']:.4f}, Diff={res['diff']:.4f}")

    max_diff = max(res['diff'] for res in results) if results else 0.0
    avg_diff = np.mean([res['diff'] for res in results]) if results else 0.0
    print(f"\nMaximum difference: {max_diff:.6f}")
    print(f"Average difference: {avg_diff:.6f}")

norm_w_aug = np.linalg.norm(w_aug)
print(f"\nNorm of initial w_aug state: {norm_w_aug:.6f} (should be 1.0 for a valid quantum state)")

print("\nVerification process complete.")

--- Verifying for 10-qubit w_aug state ---
Seed for w_aug generation (from notebook): 42
Seed for verification sampling runs: 498
Number of samples per setting for verification: 10000

Testing Pauli String: ZZZZZZZZZZ
  Theoretical Expectation: -1.000000
  Observed Expectation:    -1.000000
  Absolute Difference:     0.000000

Testing Pauli String: XXXXXXXXXX
  Theoretical Expectation: 0.000000
  Observed Expectation:    -0.003600
  Absolute Difference:     0.003600

Testing Pauli String: YYYYYYYYYY
  Theoretical Expectation: 0.000000
  Observed Expectation:    -0.003600
  Absolute Difference:     0.003600

Testing Pauli String: ZXYZXYZXYZ
  Theoretical Expectation: 0.000000
  Observed Expectation:    -0.012000
  Absolute Difference:     0.012000

Testing Pauli String: XYZXYZXYZX
  Theoretical Expectation: 0.000000
  Observed Expectation:    0.005200
  Absolute Difference:     0.005200

Testing Pauli String: ZZXXXXXXXX
  Theoretical Expectation: 0.000000
  Observed Expectation:    0.02